In [1]:
import pprint
import random
import time
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer 
from torchtext.vocab import build_vocab_from_iterator

from fetch_db import reddit_data

In [2]:
subreddit_labels = {1: "askhistorians", 2: "writingprompts", 
                 3: "television", 4: "explainlikeimfive", 
                 5: "lifeprotips", 6: "relationship_advice",
                 7: "science", 8: "books",
                 9: "nba", 10: "philosophy"}

In [3]:
tokenizer = get_tokenizer('basic_english')

In [4]:
mod_data = []
for i in reddit_data:
    title_index = list(subreddit_labels.keys())[list(subreddit_labels.values()).index(i[2])]
    mod_data.append((title_index, i[1]))
    
random.shuffle(mod_data)

In [5]:
len(mod_data)

994

In [6]:
train_iter = mod_data[:int(len(mod_data)*0.9)] 
test_iter = mod_data[int(len(mod_data)*0.9):]

In [7]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [8]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [9]:
print(vocab(["here", "is", "an", "example", "of", "some", "text"]))

[212, 12, 44, 1278, 7, 100, 489]


In [10]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [12]:
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [13]:
class TextClassificationModel(nn.Module):
    
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5 
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [14]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [15]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elased = time.time() - start_time 
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader), total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()
            
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0 
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [16]:
EPOCHS = 10 
LR = 5 
BATCH_SIZE = 64

In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None 
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.90)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                            shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val 
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.11s | valid accuracy    0.322 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.08s | valid accuracy    0.444 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.07s | valid accuracy    0.456 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.08s | valid accuracy    0.567 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.08s | valid accuracy    0.567 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.07s |

In [18]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('Test accuracy is: {:1.3f}'.format(accu_test))

Checking the results of test dataset.
Test accuracy is: 0.700


In [19]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = 'Poisoning wells was a common practice in times of war, \
               but then, at which point the well becomes drinkable again?, \
               are there well still poisoned from past wars?'

ex_text_str = 'Carl Jung\'s concept of the Collective Unconscious is often misunderstood \
               to be a collective hive consciousness but it was really a hypothesis about \
               a mental counterpart to DNA by which instinctual structures of the mind are inherited'

model = model.to("cpu")

print("This is a %s news" %subreddit_labels[predict(ex_text_str, text_pipeline)])

This is a television news
